https://leetcode.com/problems/analyze-subscription-conversion/description/

In [ ]:
-- First approach

with CTE as
(
    select user_id, activity_date, activity_type, activity_duration
    from useractivity
    where user_id in (SELECT user_id FROM useractivity GROUP BY user_id
    HAVING 
    SUM(CASE WHEN activity_type = 'free_trial' THEN 1 ELSE 0 END) > 0
    AND SUM(CASE WHEN activity_type = 'paid' THEN 1 ELSE 0 END) > 0 )
),
cte2 as 
(
    select user_id, 'free_trial' as activity_type, case when activity_type = 'free_trial' then activity_duration else NULL end as activity_duration_free_trial
    from cte
),
cte3 as 
(
     select user_id, 'paid' as activity_type, case when activity_type = 'paid' then activity_duration else NULL end as activity_duration_paid
    from cte
)
select ft.user_id, round(avg(activity_duration_free_trial),2) as trial_avg_duration,
round(avg(activity_duration_paid),2) paid_avg_duration
from cte2 ft
inner join cte3 p
on ft.user_id = p.user_id
where activity_duration_free_trial is not null and activity_duration_paid is not null
group by(ft.user_id)
order by ft.user_id
 